In [1]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import sys
sys.path.append("../../lib")
from metrics import levenshtein
import pickle

In [2]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_training_18M_without_Finnish/PL/"

In [3]:
output_folder = Path("../../data/pl")

In [4]:
files = sorted(os.listdir(folder))
len(files)

1

In [5]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

149

In [6]:
from multiprocessing import Pool

def extract(name):
    with open(name) as file:
        return file.readlines()
    
def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]
    
p = Pool(4)
    
data = list(p.imap_unordered(extract, tqdm(files), chunksize = 128))
len(data)

  0%|          | 0/149 [00:00<?, ?it/s]

149

In [7]:
# data = []
# for f in tqdm(files):
#     with open(f) as file:
#         data.append(file.readlines())

data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(149, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,"POGAŃSKICH 50 alias 0RCADES-, NERIDES, Boga de...","POGAŃSKICH 50 alias 0RCADES-, NERIDES, Boga de...","@@@@@@@@@@@@@ alias ORCADES@; NERIDES, Boga de..."
1,Difcurs o Cenie śięći/ jfjeśćio Cjero*>ttyd) 3...,@Difcurs o Cenie śięći@/ jfjeśćio Cjero*>ttyd)...,Discurs o Cenie śiⱥći / z ßeśćiⱥ Czer@@wonych...
2,V tcfii^dzytcrazpiey^ey. iaio tcjytyeiące ffar...,V tcfii^dzytcrazpiey^ey. iaio tcjytyeiące ffar...,@@@@@@@@@@@@@@@@@@@@@@@@ iáko trzytyśiⱥce stár...
3,"KATALOG AUTOROM Botera, Teatrum Świata. j er S...","@KATALOG AUTOROM Botera, Teatrum Świata. j er ...",KATALOG AUTORO@@@@@@@@@@@@@@@@@@@@@@@@@@W @& ...
4,",<U o 7 £7? MIS'ACH PAROWO MAS! YC7.NYC.fi Man...",",<U o 7 £7? MIS'ACH PAROWO MAS! YC7.NYC.fi Man...",@@@@@@@@@@@ @@@@@@H@@@@@@@@@@@@@@@@@@@@@@@@@@@...


In [8]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,149.000000,149.000000,149.000000
mean,1659.516779,1793.315436,1793.315436
std,446.693900,474.380835,474.380835
min,278.000000,285.000000,285.000000
25%,1353.000000,1480.000000,1480.000000
50%,1659.000000,1703.000000,1703.000000
75%,2005.000000,2085.000000,2085.000000
max,2624.000000,3517.000000,3517.000000


In [9]:
levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
            hypothesis = data.ocr_to_input).cer.describe()

count      149.000000
mean       256.881728
std       1304.007686
min          9.898629
25%         23.076923
50%         44.827586
75%         56.534091
max      10320.000000
Name: cer, dtype: float64

In [10]:
levenshtein(reference = data.gs_aligned, 
            hypothesis = data.ocr_aligned).cer.describe()

count    149.000000
mean      40.080196
std       19.573851
min        9.770453
25%       22.332859
50%       42.094595
75%       51.097695
max       99.278499
Name: cer, dtype: float64

In [11]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))
with open(output_folder/"data/vocabulary.pkl", "wb") as file:
    pickle.dump(vocabulary, file)

196


In [12]:
dev = data.sample(n = 5, random_state = 1)
dev.to_pickle(output_folder/"data/dev.pkl")
dev.shape

(5, 3)

In [13]:
train = data.drop(dev.index)
train.to_pickle(output_folder/"data/train.pkl")
train.shape

(144, 3)

In [14]:
train.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,144.000000,144.000000,144.000000
mean,1662.861111,1792.361111,1792.361111
std,452.602847,478.311535,478.311535
min,278.000000,285.000000,285.000000
25%,1351.750000,1478.250000,1478.250000
50%,1677.500000,1728.000000,1728.000000
75%,2033.250000,2089.250000,2089.250000
max,2624.000000,3517.000000,3517.000000


In [15]:
dev.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,5.000000,5.000000,5.000000
mean,1563.200000,1820.800000,1820.800000
std,217.827225,382.680938,382.680938
min,1364.000000,1569.000000,1569.000000
25%,1456.000000,1672.000000,1672.000000
50%,1461.000000,1674.000000,1674.000000
75%,1618.000000,1689.000000,1689.000000
max,1917.000000,2500.000000,2500.000000


In [16]:
levenshtein(reference = dev.gs_aligned.str.replace("@", ""), 
            hypothesis = dev.ocr_to_input).cer.describe()

count     5.000000
mean     52.337773
std      17.890173
min      21.951220
25%      52.176738
50%      59.146341
75%      60.416667
max      67.997898
Name: cer, dtype: float64

In [17]:
levenshtein(reference = dev.gs_aligned, 
            hypothesis = dev.ocr_to_input).cer.describe()

count     5.000000
mean     46.261210
std      14.448207
min      20.992823
25%      47.968937
50%      51.760000
75%      54.943754
max      55.640535
Name: cer, dtype: float64

In [18]:
window_length = 100

In [19]:
df = train#.head(100)
train_aligned = list(p.imap_unordered(create_windows, 
                                      tqdm(zip(df.ocr_aligned, 
                                               df.gs_aligned, 
                                               [window_length for x in df.ocr_aligned]), 
                                           total = len(df.ocr_aligned)),
                                      chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

(258244, 2)


,source,target
0,"SCT E N CY A o B O Z K ACą Jf Hycli Jowifza, G...","@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ nyc@h Jowisza, G..."
1,"CT E N CY A o B O Z K ACą Jf Hycli Jowifza, Gw...","@@@@@@@@@@@@@@@@@@@@@@@@@@@@ nyc@h Jowisza, Gw..."
2,"T E N CY A o B O Z K ACą Jf Hycli Jowifza, Gwi...","@@@@@@@@@@@@@@@@@@@@@@@@@@@ nyc@h Jowisza, Gwi..."
3,"E N CY A o B O Z K ACą Jf Hycli Jowifza, Gwia...","@@@@@@@@@@@@@@@@@@@@@@@@@@ nyc@h Jowisza, Gwia..."
4,"E N CY A o B O Z K ACą Jf Hycli Jowifza, Gwiaz...","@@@@@@@@@@@@@@@@@@@@@@@@@ nyc@h Jowisza, Gwiaz..."


In [20]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

,source,target
0,"SCT E N CY A o B O Z K ACą Jf Hycli Jowifza, G...","@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ nyc@h Jowisza, G..."
1,"CT E N CY A o B O Z K ACą Jf Hycli Jowifza, Gw...","@@@@@@@@@@@@@@@@@@@@@@@@@@@@ nyc@h Jowisza, Gw..."
2,"T E N CY A o B O Z K ACą Jf Hycli Jowifza, Gwi...","@@@@@@@@@@@@@@@@@@@@@@@@@@@ nyc@h Jowisza, Gwi..."
3,"E N CY A o B O Z K ACą Jf Hycli Jowifza, Gwia...","@@@@@@@@@@@@@@@@@@@@@@@@@@ nyc@h Jowisza, Gwia..."
4,"E N CY A o B O Z K ACą Jf Hycli Jowifza, Gwiaz...","@@@@@@@@@@@@@@@@@@@@@@@@@ nyc@h Jowisza, Gwiaz..."


In [21]:
dev_aligned = dev.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

(9109, 2)


,source,target
0,SCYFMA o BOŻKACH POGAŃSKICH linowi CNłihTl Adv...,@@@@@@@@@@@@@@@@@@@@@@@@@@@ linowi CHRISTI Aáv...
1,CYFMA o BOŻKACH POGAŃSKICH linowi CNłihTl Adve...,@@@@@@@@@@@@@@@@@@@@@@@@@@ linowi CHRISTI Aáve...
2,YFMA o BOŻKACH POGAŃSKICH linowi CNłihTl Adven...,@@@@@@@@@@@@@@@@@@@@@@@@@ linowi CHRISTI Aáven...
3,FMA o BOŻKACH POGAŃSKICH linowi CNłihTl Advent...,@@@@@@@@@@@@@@@@@@@@@@@@ linowi CHRISTI Aávent...
4,MA o BOŻKACH POGAŃSKICH linowi CNłihTl Adventu...,@@@@@@@@@@@@@@@@@@@@@@@ linowi CHRISTI Aáventu...


In [22]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

,source,target
0,SCYFMA o BOŻKACH POGAŃSKICH linowi CNłihTl Adv...,@@@@@@@@@@@@@@@@@@@@@@@@@@@ linowi CHRISTI Aáv...
1,CYFMA o BOŻKACH POGAŃSKICH linowi CNłihTl Adve...,@@@@@@@@@@@@@@@@@@@@@@@@@@ linowi CHRISTI Aáve...
2,YFMA o BOŻKACH POGAŃSKICH linowi CNłihTl Adven...,@@@@@@@@@@@@@@@@@@@@@@@@@ linowi CHRISTI Aáven...
3,FMA o BOŻKACH POGAŃSKICH linowi CNłihTl Advent...,@@@@@@@@@@@@@@@@@@@@@@@@ linowi CHRISTI Aávent...
4,MA o BOŻKACH POGAŃSKICH linowi CNłihTl Adventu...,@@@@@@@@@@@@@@@@@@@@@@@ linowi CHRISTI Aáventu...


In [23]:
train_aligned.to_pickle(output_folder/"data/train_aligned.pkl")
dev_aligned.to_pickle(output_folder/"data/dev_aligned.pkl")